<a href="https://colab.research.google.com/github/Tezashree/DataScienceBootcamp/blob/main/Projects/titanic_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
df = pd.read_csv("titanic_Train.csv")
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [59]:
# Count the number of males and females
sex_counts = df['Sex'].value_counts()

print("Number of males:", sex_counts['male'])
print("Number of females:", sex_counts['female'])


# Male and Female Survived
survived_counts = df.groupby('Sex')['Survived'].sum()
print("Number of males and females who survived:")
print(survived_counts)


Number of males: 577
Number of females: 314
Number of males and females who survived:
Sex
female    233
male      109
Name: Survived, dtype: int64


In [60]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [61]:
# Max,Min,Avg.Price

class_stats = df.groupby('Pclass')['Fare'].agg(['max', 'min', 'mean'])

print(class_stats)

             max  min       mean
Pclass                          
1       512.3292  0.0  84.154687
2        73.5000  0.0  20.662183
3        69.5500  0.0  13.675550


In [62]:
# Age of survived passengers
bins = [0, 14, 30, 150]
labels = ['child', 'adult', 'old']
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
survivors_age = df[df['Survived'] == 1]['AgeGroup'].value_counts()
print(survivors_age)

old      134
adult    114
child     42
Name: AgeGroup, dtype: int64


In [63]:
# People  starting their journey
embark_counts = df['Embarked'].value_counts()
gender_counts = df.groupby('Embarked')['Sex'].value_counts().unstack()

print("Number of people embarking from each port:")
print(embark_counts)

print("\nNumber of males and females embarking from each port:")
print(gender_counts)

Number of people embarking from each port:
S    644
C    168
Q     77
Name: Embarked, dtype: int64

Number of males and females embarking from each port:
Sex       female  male
Embarked              
C             73    95
Q             36    41
S            203   441


In [64]:
# Removing columns
print(df.columns)
remove_columns = ["PassengerId", "Name", "Ticket"]
df = df.drop(remove_columns,axis=1)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'AgeGroup'],
      dtype='object')


In [65]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
AgeGroup    177
dtype: int64

In [66]:
print(df.shape)

(891, 10)


In [67]:
df = df.drop(columns=["Cabin", "AgeGroup"])


In [68]:
df = df.dropna(subset=["Embarked"])

In [69]:
#fill Nan value in the  age column with mean
mean_age = df["Age"].mean()
df['Age'] = df['Age'].fillna(mean_age)
print(df)

     Survived  Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0           0       3    male  22.000000      1      0   7.2500        S
1           1       1  female  38.000000      1      0  71.2833        C
2           1       3  female  26.000000      0      0   7.9250        S
3           1       1  female  35.000000      1      0  53.1000        S
4           0       3    male  35.000000      0      0   8.0500        S
..        ...     ...     ...        ...    ...    ...      ...      ...
886         0       2    male  27.000000      0      0  13.0000        S
887         1       1  female  19.000000      0      0  30.0000        S
888         0       3  female  29.642093      1      2  23.4500        S
889         1       1    male  26.000000      0      0  30.0000        C
890         0       3    male  32.000000      0      0   7.7500        Q

[889 rows x 8 columns]


In [70]:
#one-hot-encoding for categorical features
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
categorical_features = ["Sex", "Embarked"]
df_encoded = pd.get_dummies(df, columns=categorical_features)

#one-hot-encoding for Survived label
label_encoder = OneHotEncoder()
labels_encoded = label_encoder.fit_transform(df[['Survived']])

In [71]:
#perform min,max Scalaing for Numerical columns
scaler = MinMaxScaler()
numerical_columns = ['Age', 'Fare']
df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])

In [72]:
#StandardScalar
from sklearn.preprocessing import OneHotEncoder, StandardScaler
scaler = StandardScaler()
numerical_columns = ['Age', 'Fare']
df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])

In [73]:
#Train_TestSplit
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop('Survived', axis=1), labels_encoded.toarray(), test_size=0.2, random_state=42)


In [74]:
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn_classifier= KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_score = knn_classifier.score(X_test,y_test)
print(knn_score)

0.7921348314606742


In [75]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
randomforest_classifier = RandomForestClassifier()
randomforest_classifier.fit(X_train, y_train)
randomforest_score = randomforest_classifier.score(X_test, y_test)
print(randomforest_score)

0.7808988764044944


In [76]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree_classifier = DecisionTreeClassifier()
decisiontree_classifier.fit(X_train, y_train)
decisiontree_score = decisiontree_classifier.score(X_test, y_test)
print(decisiontree_score)

0.7247191011235955
